In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_union, make_pipeline, Pipeline
from sklearn.model_selection import ParameterGrid

from sklearn.impute import MissingIndicator, KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler

from fenginering.indicator import *
from fenginering.transformer import *
from data.get_data import *

%load_ext autoreload
%autoreload 2

In [ ]:
sma_list = [3, 9, 12]
ema_list = [3, 9, 15]
rsi_list = [10, 14, 20]
atr_list = [3, 6, 9]
adx_list = [3, 6, 9]

n_day_up_list = [10, 15, 20]
aroon_list = [5, 10, 15]

In [ ]:
stochastic_oscillator_dict = {
    'k_period' : [10, 15, 20],
    'd_period' : [3, 5]
}
bande_bollingers_dict = {
    'window' : [10, 15, 20],
    'wind_dev' : [3, 5]
}
macd_dict = {
    'slow' : [10, 14, 20],
    'fast' : [3, 5, 8],
    'signal' : [3, 4]
}

stochastic_oscillator_grid = param_grid(stochastic_oscillator_dict)
bande_bollingers_grid = param_grid(bande_bollingers_dict)
macd_grid = param_grid(macd_dict)

In [ ]:
SMA_transformer = one_params_transformer(sma_list, sma)
EMA_transformer = one_params_transformer(ema_list, ema)
RSI_transformer = one_params_transformer(rsi_list, rsi)
N_day_up_transformer = one_params_transformer(n_day_up_list, n_day_up)
ATR_transformer = one_params_transformer(atr_list, atr)
ADX_transformer = one_params_transformer(adx_list, adx)
SAR_transformer = zero_params_transformer(sar)

AROON_transformer = one_params_transformer(aroon_list, aroon)


STOCHASTIQUE_OSCILLATOR_transformer = multi_params_transformer(stochastic_oscillator_grid, stochastic_oscillator)
BANDE_BOLLINGERS_transformer = multi_params_transformer(bande_bollingers_grid, bande_bollingers)
MACD_transformer = multi_params_transformer(macd_grid, macd)

In [ ]:
SAR = make_pipeline(
    SAR_transformer,
    SimpleImputer(strategy = 'constant',
                  fill_value = 0)
)
SAR
#impute_sar.fit_transform(data)

In [ ]:
SAR_transformer.

In [ ]:
SAR.feature_names_out
#SAR_transformer.get_feature_names_out()

In [ ]:
#SAR.fit_transform(data)

In [ ]:
SAR_transformer.fit_transform(data)


In [ ]:
SAR_transformer.feature_names_out = ['a', 'b', 'c']

In [ ]:
SAR_transformer.get_feature_names_out()

In [ ]:
SAR.fit_transform(data)

In [ ]:
pipeline_indicator = make_union(
    #SMA_transformer,
    EMA_transformer,
    RSI_transformer,
    N_day_up_transformer,
    ATR_transformer,
    ADX_transformer,
    #SAR,
    AROON_transformer,
    STOCHASTIQUE_OSCILLATOR_transformer,
    BANDE_BOLLINGERS_transformer,
    MACD_transformer
)


pipeline_indicator


In [ ]:
DROP_NA_transformer = dropna_transformer()

data_pipeline = make_pipeline(
    pipeline_indicator,
    DROP_NA_transformer
)
data_pipeline

In [ ]:
start = "2022"
end = "2023"
data = get_data('BTC', '1d').loc[start : end]
data.drop(columns = ['symbol'], inplace = True)
data.columns = data.columns.astype(str)
data.shape

In [ ]:
#data_pipeline.fit_transform(data)

# Model

In [ ]:
from preprocessing.function import *
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

N = data.shape[0]
n = int(0.7*N)
train = data[:n]
test = data[n:]


target = get_classification_target(train)
features = train

target_test = get_classification_target(test)
features_test = test

target.value_counts().plot(kind = 'barh', color = 'red')

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
model = RandomForestClassifier()
model.fit(features, target)

In [ ]:
y_pred = model.predict(test)
y_pred_proba = model.predict_proba(test)

from evaluation.classification import *

evaluation = Evaluation(y_true = target_test, y_pred = y_pred, proba_pred = y_pred_proba[:,1])
evaluation.metrics()
evaluation.show()

In [ ]:
Importance = pd.DataFrame({'Importance' : model.feature_importances_*100},
                          index = model.feature_names_in_)
Importance.sort_values('Importance', axis = 0, ascending = True).plot(kind = 'barh', color = 'r')